# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [127]:
# import libraries
import findspark
findspark.init()
findspark.find()

import pyspark
findspark.find()

'C:\\spark\\spark-2.4.5-bin-hadoop2.7'

In [128]:
# create a Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Sparkify').getOrCreate()

In [129]:
# functions to import
from pyspark.sql.functions import count, udf
import pyspark.sql.functions as func
import datetime
from time import time

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [130]:
# Read in data
df = spark.read.json('medium-sparkify-event-data.json')

In [131]:
df.head(2)

[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293'),
 Row(artist="John Brown's Body", auth='Logged In', firstName='Sawyer', gender='M', itemInSession=74, lastName='Larson', length=380.21179, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=97, song='Bulls', status=200, ts=1538352025000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98')]

In [132]:
# This is not necessarily the schema we want
# For e.g. 'long' data types may make some things hard
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [133]:
df.head(2)

[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293'),
 Row(artist="John Brown's Body", auth='Logged In', firstName='Sawyer', gender='M', itemInSession=74, lastName='Larson', length=380.21179, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=97, song='Bulls', status=200, ts=1538352025000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98')]

In [134]:
df.count()

543705

In [135]:
# if count != 286500, this may mean that these columns having missing values
df.describe(['artist','firstName','gender','lastName','length','location']).show()

+-------+-----------------+---------+------+--------+-----------------+----------------+
|summary|           artist|firstName|gender|lastName|           length|        location|
+-------+-----------------+---------+------+--------+-----------------+----------------+
|  count|           432877|   528005|528005|  528005|           432877|          528005|
|   mean|527.5289537712895|     null|  null|    null|248.6645927800741|            null|
| stddev|966.1072451772757|     null|  null|    null|98.41266955052022|            null|
|    min|              !!!|  Aaliyah|     F|  Abbott|          0.78322|       Akron, OH|
|    max|ÃÂlafur Arnalds|   Zyonna|     M|  Wright|       3024.66567|York-Hanover, PA|
+-------+-----------------+---------+------+--------+-----------------+----------------+



In [136]:
# if count != 286500, this may mean that these columns having missing values
df.describe(['registration','song','userAgent','userID']).show()

+-------+--------------------+--------------------+--------------------+------------------+
|summary|        registration|                song|           userAgent|            userID|
+-------+--------------------+--------------------+--------------------+------------------+
|  count|              528005|              432877|              528005|            543705|
|   mean|1.535523414863953...|            Infinity|                null| 60268.42669103512|
| stddev|3.0787254929962535E9|                 NaN|                null|109898.82324176558|
|    min|       1509854193000|ÃÂg ÃÂtti Gr...|"Mozilla/5.0 (Mac...|                  |
|    max|       1543073874000|ÃÂ¾etta Gerist Ã...|Mozilla/5.0 (comp...|                99|
+-------+--------------------+--------------------+--------------------+------------------+



In [137]:
'''
It appears that UserID and SessionID don't have missing values
But why is the 'min' of userID = 'blank'

''' 
df.describe(['userID','sessionID']).show()

+-------+------------------+------------------+
|summary|            userID|         sessionID|
+-------+------------------+------------------+
|  count|            543705|            543705|
|   mean| 60268.42669103512|2040.8143533717732|
| stddev|109898.82324176558|1434.3389310782666|
|    min|                  |                 1|
|    max|                99|              4808|
+-------+------------------+------------------+



In [138]:
# Check count of records where userID is blank
df.filter("userID == ''").count()

15700

In [139]:
# Check count of records where userID is blank
df.filter("sessionID == ''").count()

0

In [140]:
# Only keep rows that have userID
df = df.where("userID != ''")

In [141]:
df.count()

528005

In [142]:
'''
Convert fields with timestamps to Date and Timestamp
'''
convert_ts = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%Y-%m-%d %H:%M:%S"))
df = df.withColumn('event_time', convert_ts('ts'))
df = df.withColumn('registration_time', convert_ts('registration'))

In [143]:
df.head(10)

[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293', event_time='2018-09-30 17:00:11', registration_time='2018-07-19 22:11:47'),
 Row(artist="John Brown's Body", auth='Logged In', firstName='Sawyer', gender='M', itemInSession=74, lastName='Larson', length=380.21179, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=97, song='Bulls', status=200, ts=1538352025000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98', e

In [144]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- registration_time: string (nullable = true)



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

In [145]:
'''
Notes
- Cancellation Confirmation & Downgrade -> 'Page' column
- Paid vs Free users -> 'Level' column

For reading
- https://clevertap.com/blog/churn-rate/

'''

"\nNotes\n- Cancellation Confirmation & Downgrade -> 'Page' column\n- Paid vs Free users -> 'Level' column\n\nFor reading\n- https://clevertap.com/blog/churn-rate/\n\n"

In [146]:
df_full = df.withColumn('churn', func.when(func.col('page') == 'Cancellation Confirmation', 1).otherwise(0))

In [147]:
df_full = df_full.withColumn('downgrade', func.when(func.col('page') == 'Downgrade', 1).otherwise(0))

In [148]:
df_full.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- registration_time: string (nullable = true)
 |-- churn: integer (nullable = false)
 |-- downgrade: integer (nullable = false)



### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [149]:
df_full.groupBy("level").count().show()

+-----+------+
|level| count|
+-----+------+
| free|109961|
| paid|418044|
+-----+------+



In [150]:
df_full.groupBy("page").count().show()

+--------------------+------+
|                page| count|
+--------------------+------+
|              Cancel|    99|
|    Submit Downgrade|   117|
|         Thumbs Down|  4911|
|                Home| 19089|
|           Downgrade|  3811|
|         Roll Advert|  7773|
|              Logout|  5990|
|       Save Settings|   585|
|Cancellation Conf...|    99|
|               About|  1026|
|            Settings|  2964|
|     Add to Playlist| 12349|
|          Add Friend|  8087|
|            NextSong|432877|
|           Thumbs Up| 23826|
|                Help|  2644|
|             Upgrade|   968|
|               Error|   503|
|      Submit Upgrade|   287|
+--------------------+------+



In [151]:
df_full.groupBy("status").count().show()

+------+------+
|status| count|
+------+------+
|   307| 43902|
|   404|   503|
|   200|483600|
+------+------+



In [152]:
df_full.groupBy("churn").count().show()

+-----+------+
|churn| count|
+-----+------+
|    1|    99|
|    0|527906|
+-----+------+



In [153]:
df_full.groupBy("downgrade").count().show()

+---------+------+
|downgrade| count|
+---------+------+
|        1|  3811|
|        0|524194|
+---------+------+



In [154]:
churn_users = df_full.select(['userID',
                'gender',
                'level',
                'registration_time',
                'event_time']
              ).filter(df_full["churn"]==1)

In [155]:
# Add column that calculates duration (in days) between the churn event and user registration event
churn_users = churn_users.withColumn('member_duration', ((func.unix_timestamp(churn_users['event_time']) - func.unix_timestamp(churn_users['registration_time']))/3600)/24)

In [162]:
churn_users.show()

+------+------+-----+-------------------+-------------------+------------------+
|userID|gender|level|  registration_time|         event_time|   member_duration|
+------+------+-----+-------------------+-------------------+------------------+
|   208|     F| free|2018-09-12 06:20:39|2018-10-01 06:30:16| 19.00667824074074|
|   231|     F| paid|2018-08-05 05:38:20|2018-10-02 05:19:53|57.987187500000005|
|   236|     M| paid|2018-06-25 23:46:19|2018-10-02 21:11:11| 98.89226851851852|
|   156|     M| free|2018-08-10 00:53:40|2018-10-03 11:48:49|54.454965277777774|
|   271|     M| paid|2018-08-30 01:51:34|2018-10-05 13:07:57|36.469710648148144|
|   200|     M| paid|2018-09-28 23:17:12|2018-10-05 21:35:31| 6.929386574074074|
|   130|     M| free|2018-08-06 10:07:16|2018-10-06 22:41:35|61.523831018518514|
|   123|     M| free|2018-08-15 15:08:03|2018-10-07 17:38:01| 53.10414351851852|
|    54|     F| paid|2018-07-24 18:37:42|2018-10-08 01:33:06| 75.28847222222223|
|    39|     F| paid|2018-06

In [163]:
# Average duration before church is 57 days
churn_users.describe(['member_duration']).show()

+-------+-------------------+
|summary|    member_duration|
+-------+-------------------+
|  count|                 99|
|   mean|   67.9680198980546|
| stddev|  43.54582091999607|
|    min|-0.5227314814814815|
|    max| 263.91400462962963|
+-------+-------------------+



In [164]:
# Most churns are males
churn_users.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|   45|
|     M|   54|
+------+-----+



In [165]:
# Most churns are paid users
churn_users.groupBy("level").count().show()

+-----+-----+
|level|count|
+-----+-----+
| free|   32|
| paid|   67|
+-----+-----+



In [166]:
''' 
Analyze specific churn user

'''
df_full.select(['sessionId','length','level','page','status','event_time']).filter(df_full["userID"]=='32').orderBy('event_time', ascending=False).show()

+---------+---------+-----+--------------------+------+-------------------+
|sessionId|   length|level|                page|status|         event_time|
+---------+---------+-----+--------------------+------+-------------------+
|     1881|     null| paid|Cancellation Conf...|   200|2018-10-17 20:04:56|
|     1881|     null| paid|              Cancel|   307|2018-10-17 20:03:48|
|     1881|     null| paid|           Downgrade|   200|2018-10-17 20:03:47|
|     1881|348.29016| paid|            NextSong|   200|2018-10-17 20:02:46|
|     1881|304.84853| paid|            NextSong|   200|2018-10-17 19:57:42|
|     1881|293.56363| paid|            NextSong|   200|2018-10-17 19:52:49|
|     1881|181.75955| paid|            NextSong|   200|2018-10-17 19:49:48|
|     1881|343.71873| paid|            NextSong|   200|2018-10-17 19:44:05|
|     1881|185.44281| paid|            NextSong|   200|2018-10-17 19:41:00|
|     1881|239.17669| paid|            NextSong|   200|2018-10-17 19:37:01|
|     1881|3

In [167]:
user_32 = df_full.filter(df_full["userID"]=='32')
user_32.groupBy("page").count().show()

+--------------------+-----+
|                page|count|
+--------------------+-----+
|              Cancel|    1|
|         Thumbs Down|    1|
|                Home|   15|
|           Downgrade|    4|
|         Roll Advert|    7|
|              Logout|    4|
|Cancellation Conf...|    1|
|            Settings|    1|
|     Add to Playlist|   11|
|          Add Friend|    1|
|            NextSong|  201|
|           Thumbs Up|    9|
|                Help|    2|
|             Upgrade|    2|
|      Submit Upgrade|    1|
+--------------------+-----+



In [168]:
user_32.groupBy("status").count().show()

+------+-----+
|status|count|
+------+-----+
|   307|   17|
|   200|  244|
+------+-----+



In [169]:
user_32.groupBy("level").count().show()

+-----+-----+
|level|count|
+-----+-----+
| free|  141|
| paid|  120|
+-----+-----+



In [170]:
df_full.select("userID").distinct().count()

448

In [ ]:
'''
To Do
- Get # of sessionID per user
- Get Avg / Total duration of sessionID per user
- Get # of errors
- Get total member duration
- Get Avg 'Next Song' per session
- Get Total 'Next Song'
- Get # of Downgrades
- Get # of thumbs up and thumbs down
- Something about roll adverts?
- Get 'level' information
- Get # of days free vs paid

--> How to apply One Hot Encoding?


'''

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.